Analyse des données

Préparation des données avec pandas


In [34]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Dataset name_basics

In [35]:
df = import_datasets("clean_datasets/movies.csv", types="pandas", sep=",")

2023-10-23 16:08:56 INFO     Pandas loaded ! Importing movies.csv...


In [36]:
print(df.dtypes)

titre_id              object
titre_str             object
titre_type            object
titre_date_sortie    float64
titre_date_fin        object
titre_duree           object
titre_genres          object
person_id             object
person_name           object
person_birthdate      object
person_job            object
person_role           object
person_index           int64
dtype: object


In [37]:
cleaning = DataCleaner()
test = cleaning.fix_values(df, "fix_n")
test1 = cleaning.fix_values(test, "fix_encode")
clean_porn = cleaning.clean_porn(test1)
logging.info(f"Cleaned : {len(df) - len(clean_porn)} rows")

2023-10-23 16:09:03 INFO     Fixing N values...
2023-10-23 16:09:07 INFO     Fixing encoding values...
2023-10-23 16:09:20 INFO     Cleaning porn movies...
2023-10-23 16:09:23 INFO     Cleaned : 477069 rows


In [38]:
# print(len(yy[yy["titre_date_sortie"].isna()]))
# yy["titre_date_sortie"].fillna(1678, inplace=True)
# yy["person_birthdate"] = yy["person_birthdate"].astype(int).replace(0, 1678)

In [39]:
# condi = (
#     yy["person_birthdate"] < 1678
# )

# yy.loc[condi, "person_birthdate"] = 1678

In [40]:
# datime_col = ["titre_date_sortie", "person_birthdate"]
# for time in datime_col:
#     yy[time] = pd.to_datetime(yy[time], format="%Y")

In [41]:
# yy["titre_date_sortie"] = yy["titre_date_sortie"].dt.year

In [42]:
"""
Nettoyage des date_sorties nan et modification de la colonne.
Nous avons plusieurs methodes disponible, comme:

clean_porn['titre_date_sortie'].fillna(-1, inplace=True)
clean_porn.dropna(subset=['titre_date_sortie'], inplace=True)
et une fois que l'on a clean
on applique int a toute la colonne
clean_porn['titre_date_sortie'] = clean_porn['titre_date_sortie'].astype(int)

"""
clean_porn['titre_date_sortie'].fillna(0, inplace=True)
clean_porn['titre_date_sortie'] = clean_porn['titre_date_sortie'].astype("int64")
clean_porn['person_birthdate'] = clean_porn['person_birthdate'].astype("int64")
clean_porn['titre_duree'] = clean_porn['titre_duree'].astype("int64")

line = "clean_datasets/movies_clean.csv"
clean_porn.to_csv(line, index=False)

In [43]:
condi = (
    clean_porn["person_job"].str.contains("act")
)
df = clean_porn[condi]
actor = df.copy()

"""
La moyenne d'age des acteurs a la date de sortie du film.
Nous calculons la mediane sur cette base.

other KPI, calcul la moyenne des ages en fonction de l'année

"""
# ajouter colonne age pour les personnes
actor["person_age"] = np.where(
    (actor["person_birthdate"] > 1800) &
    (actor["titre_date_sortie"] > 1800) &
    (actor["titre_date_sortie"] - actor["person_birthdate"] >= 0),
    (actor["titre_date_sortie"] - actor["person_birthdate"]),
    0
)
actor.tail()

,titre_id,titre_str,titre_type,titre_date_sortie,titre_date_fin,titre_duree,titre_genres,person_id,person_name,person_birthdate,person_job,person_role,person_index,person_age
4609226,tt9916706,Dankyavar Danka,movie,2013,0,0,Comedy,nm3272130,Ashwini Ekbote,1972,actress,0,4,41
4609232,tt9916730,6 Gunn,movie,2017,0,116,Drama,nm6096005,Devadhar Archit,0,actor,0,1,0
4609233,tt9916730,6 Gunn,movie,2017,0,116,Drama,nm0059461,Sunil Barve,0,actor,0,2,0
4609234,tt9916730,6 Gunn,movie,2017,0,116,Drama,nm13233318,Ganesh Vasant Patil,0,actor,"[""Vishal""]",3,0
4609235,tt9916730,6 Gunn,movie,2017,0,116,Drama,nm4852679,Bhushan Pradhan,0,actor,0,4,0


In [44]:
"""
imagine un fonction, qui prendrais en argument une liste de date de
titre de sortie et retourne la moyenne d'age de ces années des acteurs.

# Quels sont les acteurs les plus présents ?
# À quelle période ?
La durée moyenne des films s'allonge ou se raccourcit avec les années ?
Les acteurs de série sont-ils les mêmes qu'au cinéma ?
# Les acteurs ont en moyenne quel âge ?
Quels sont les films les mieux notés ?
Partagent-ils des caractéristiques communes ? etc...

"""
bins, names = bins_generator(actor["titre_date_sortie"].max())

actor["cuts"] = pd.cut(
    actor["titre_date_sortie"],
    bins=bins,
    labels=names
)

for date in names[1:]:
    condi = (
        (actor["person_age"] != 0) &
        (actor["cuts"] == date)
    )
    print(f"Mediane for person_age {date} :", actor["person_age"][condi].median())


Mediane for person_age 1900-1920 : 35.0
Mediane for person_age 1920-1940 : 33.0
Mediane for person_age 1940-1960 : 35.0
Mediane for person_age 1960-1980 : 36.0
Mediane for person_age 1980-1990 : 38.0
Mediane for person_age 1990-2000 : 37.0
Mediane for person_age 2000-2010 : 37.0
Mediane for person_age 2010-2020 : 40.0
Mediane for person_age >2021 : 45.0


In [45]:
# Top 5 des acteurs all time !
print(actor["person_name"].value_counts().head(5))
# Bottom 5 des acteurs all time !
print(actor["person_name"].value_counts().tail(5))

person_name
Brahmanandam          729
Eddie Garcia          436
Paquito Diaz          356
Eric Roberts          347
Mithun Chakraborty    334
Name: count, dtype: int64
person_name
Audrey Wagner Elam     1
Ivy Vega               1
Jimilee Allen          1
Albert de Jong         1
Ganesh Vasant Patil    1
Name: count, dtype: int64


In [46]:
# Top 5 actors by years
for date in names[1:]:
    top5 = actor['person_name'][condi].value_counts().head(5)
    condi = (
        (actor["cuts"] == date)
    )
    top5_str = '\n'.join(str(top5).split('\n')[:-1])

    print(f"Top 5 actors by movies for {date}\n{'-'*20}\n{top5_str}")
    print("-"*20)


Top 5 actors by movies for 1900-1920
--------------------
person_name
Jason Young        150
Eric Roberts        77
Claudette Young     43
Lee Jagow           32
Susan Jarrett       31
--------------------
Top 5 actors by movies for 1920-1940
--------------------
person_name
Joseph J. Dowling    71
Olga Engl            69
Theodore Roberts     66
Frederik Jacobsen    62
William Russell      60
--------------------
Top 5 actors by movies for 1940-1960
--------------------
person_name
Akane Hisano      172
Rintarô Fujima    155
Sau-Nin Wong      147
Itoji Koto        137
Jun Arai          133
--------------------
Top 5 actors by movies for 1960-1980
--------------------
person_name
Chhabi Biswas        108
Mahmoud Al Meleji    102
Andrés Soler          97
Luis Aguilar          96
Ahmet Tarik Tekçe     91
--------------------
Top 5 actors by movies for 1980-1990
--------------------
person_name
Seong-il Shin      295
Eddie Garcia       220
Paquito Diaz       219
Cüneyt Arkin       204
Yeon

In [47]:
#  World record of the longest documentary
# source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
longest = actor[actor["titre_duree"] == 43200]
n = longest[["titre_str", "titre_duree"]].iloc[0]
heure = n["titre_duree"] // 60
jours = heure // 24
print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# All time median for duration
print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")


Le film le plus long est Ambiancé d'une durée de 720 heures soit 30 jours
All time duration median : 84 minutes


In [48]:
# Median duration for movies between years.
for date in names[1:]:
    condi = (
        (actor["cuts"] == date)
    )
    med = round(actor["titre_duree"][condi].median())
    print(f"Median durations for movies {date}\nMedian for this years is {med} minutes")
    print("-"*20)


Median durations for movies 1900-1920
Median for this years is 50 minutes
--------------------
Median durations for movies 1920-1940
Median for this years is 61 minutes
--------------------
Median durations for movies 1940-1960
Median for this years is 84 minutes
--------------------
Median durations for movies 1960-1980
Median for this years is 87 minutes
--------------------
Median durations for movies 1980-1990
Median for this years is 90 minutes
--------------------
Median durations for movies 1990-2000
Median for this years is 90 minutes
--------------------
Median durations for movies 2000-2010
Median for this years is 90 minutes
--------------------
Median durations for movies 2010-2020
Median for this years is 87 minutes
--------------------
Median durations for movies >2021
Median for this years is 80 minutes
--------------------
